In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## GPU配置

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## 超参数

In [3]:
input_size = 784 # 28*28
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

## MNIST 数据集

In [4]:
train_dataset = torchvision.datasets.MNIST(root='../../data/MNIST',
                                          train=False,
                                          transform=transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root='../../data/MNIST',
                                         train=False,
                                         transform=transforms.ToTensor())

## 数据生成器Data Loader

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=64,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                         batch_size = 64,
                                         shuffle=False)

## 构建带一个隐藏层的全连层网络

In [7]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        #首先找到test的父类（比如是类A），然后把类test的对象self转换位类A的对象，然后“被转换”的类A对象调用自己的__init__
        #即化身父类，调用__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

## 损失和优化器

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 训练模型

In [9]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # 将张量转移到配置的GPU
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        #前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)
        #反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/157], Loss: 0.3059
Epoch [2/5], Step [100/157], Loss: 0.2652
Epoch [3/5], Step [100/157], Loss: 0.0684
Epoch [4/5], Step [100/157], Loss: 0.0709
Epoch [5/5], Step [100/157], Loss: 0.0257


## 测试模型

In [12]:
# 这里我们不用计算梯度（为了内存利用效率）
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()#item方法若为单元素tensor，则返回pyton的scalar（标量）
        #print(type((predicted == labels).sum()))---------<class 'torch.Tensor'>
        
    print('Accuracy of the network on the 10000 test images: {}%'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.1%


## 保存模型

In [13]:
torch.save(model.state_dict(), 'model.ckpt')